# Webscrape college applications

This notebook was developed to work with pdfs scraped with `ColAppScrape.ipynb`.


In [1]:
import re, string, unicodedata
import os, sys
import nltk
import contractions
import inflect
import PyPDF2
import pandas as pd
from pandas import Series, DataFrame
from operator import itemgetter
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

## Define Helper Functions

In [2]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def stem_words(words):
    # Stem words in list of tokenized words
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    # Lemmatize verbs in list of tokenized words
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def remove_pmarks(text):
    text = re.sub(r'~|`|:|;|"|,|=|-|_|#', '', text)
    text = str.replace(text, '"', '')
    text = str.replace(text, "'", '')
    text = str.replace(text, '.', '')
    text = str.replace(text, '?', '')
    text = str.replace(text, ')', '')
    text = str.replace(text, '(', '')
    text = str.replace(text, '|', '')
    text = str.replace(text, '$', '')
    text = str.replace(text, '%', '')
    return text

def remove_nontxt(text):
    text = str.replace(text, r'\\n', '')
    text = str.replace(text, r'\n', '')
    return(text)

def remove_common_words(text):
    words_to_drop = ['a', 'all', 'an', 'and', 'are', 'as', 'at', 'be', 'but',
                     'by', 'can', 'for', 'i', 'if', 'in', 'is', 'it', 
                     'my', 'not', 'of', 'on', 'or', 'that', 
                     'the', 'to', 'will', 'with', 'you']
    for word in words_to_drop:
        text = re.sub(''.join((r'\b', word, r'\b')), '', text)
    return text

def combine_cword(text):
    compound_words = {
        'social security number': 'socialsecuritynumber',
        'ssn': 'socialsecuritynumber',
        'high school': 'highschool'
    }
    for cword in compound_words:
        text = re.sub(''.join((r'\b', cword, r'\b')), compound_words[cword], text)
    return text

def get_words(text):
    return nltk.word_tokenize(text)

def remove_non_ascii(words):
    # Remove non-ASCII characters from list of tokenized words
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def replace_numbers(words):
    # Replace all interger occurrences in list of tokenized words with textual representation
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_numbers(words):
    # Remove all intergers
    new_words = []
    for word in words:
        if word.isdigit() == False:
            new_words.append(word)
    return new_words

def replace_nolen(words):
    # Replcae words that are no length (i.e. '')
    new_words = []
    for word in words:
        if len(word) > 0:
            new_words.append(word)
    return new_words

def normalize(text):
    # Remove punctuation from entire string.
    # print('Remove punctuation.', end='')
    text = remove_pmarks(text)
    # print(' DONE')
    # Remove non text
    # print('Remove non text....', end='')
    text = remove_nontxt(text)
    # print(' DONE')
    # Put to lowercase first
    # print('Converting case....', end='')
    text = text.lower()
    # print(' DONE')
    # Remove unwanted common words.
    # print('Remove freq words..', end='')
    text = remove_common_words(text)
    # print(' DONE')
    # Handle common application compound words
    # print('Handle compounds...', end='')
    text = combine_cword(text)
    # print(' DONE')
    # Tokenize the string.
    # print('Tokenize string....', end='')
    words = get_words(text)
    # print(' DONE')
    # Remove ascii characters.
    # print('Remove non ascii...', end='')
    words = remove_non_ascii(words)
    # print(' DONE')
    # Convert numebrs to words.
    # print('Replace numbers....', end='')
    # words = replace_numbers(words)
    # print(' DONE')
    # Convert numebrs to words.
    # print('Remove numbers.....', end='')
    words = remove_numbers(words)
    # print(' DONE')
    # Remove zero length words.
    # print('Remove zero length.', end='')
    words = replace_nolen(words)
    # print(' DONE')
    return words

def get_sorted_count(words):
    index_list = sorted(set(words))
    count_list = list(range(len(index_list)))
    for i in range(len(index_list)):
        count_list[i] = 0
        for word in words:
            if word == index_list[i]:
                count_list[i] = count_list[i] + 1
    grand_list = []
    for i in range(len(index_list)):
        item_list = []
        item_list.append(index_list[i])
        item_list.append(count_list[i])
        grand_list.append(item_list)
    return sorted(grand_list, key=itemgetter(1), reverse=True)

In [31]:
testlist = ['hihi','cat','dog']
holder = get_sorted_count(testlist)
print(holder)
print(holder.append(['new',2]))

holder.append(['new',2])
holder


[['cat', 1], ['dog', 1], ['hihi', 1]]
None


[['cat', 1], ['dog', 1], ['hihi', 1], ['new', 2], ['new', 2]]

In [33]:
testlist = ['hihi','cat','dog']
holder = get_sorted_count(testlist).append(['new',2])
holder

# Implement Text Processing

In [63]:
all_apps_df = DataFrame([], columns=['word','fq0'])

In [ ]:
test_file = open(os.path.join('pprapps', 'carver.edu2.pdf'), 'rb')
test_pdf = PyPDF2.PdfFileReader(test_file)
test_pdf.numPages

In [64]:
import time
from tqdm import tqdm_notebook

# PyPDF2 has an undocumented PdfReadWarning
# (PdfReadWarning: Superfluous whitespace found in object header b'1' b'0')
# Example: Throws this warning for carver.edu2.pdf (among others)
# https://stackoverflow.com/questions/5644836/in-python-how-does-one-catch-warnings-as-if-they-were-exceptions/39077786
# import warnings
# warnings.filterwarnings("error")

error_log = []

# for file in tqdm_notebook(os.listdir(os.path.join('pprapps'))[224:228]):
# for file in os.listdir(os.path.join('pprapps'))[:3]:
for file in tqdm_notebook(os.listdir(os.path.join('pprapps'))[0:1]):
    print(file)
    try:
        print('a')
        app_file = PyPDF2.PdfFileReader(
            open(os.path.join('pprapps', file), 'rb'))
    except:
        print('b')
        error_log.append('Error loading : ' + os.path.join(
            'pprapps', file))
    
    # num_of_pgs = app_file.numPages
    try:
        print('c')
        num_of_pgs = app_file.numPages
    except:
        print('d')
        num_of_pgs = 100
        error_log.append('Error getting no of pgs : ' + os.path.join(
            'pprapps', file))
        
    if num_of_pgs < 20:
        print('e')
        app_file_text = []
        for pageNum in range(app_file.numPages):
            app_file_text.append(
                app_file.getPage(pageNum).extractText().encode(
                    'utf-8', 'ignore').decode('utf-8', 'ignore'))
        # Use file name to create an unique header
        f_col = str.replace(file, 'pdf', '')
        f_col = f_col[:-1]
        
        app_file_listed = get_sorted_count(
            normalize(str(app_file_text)))
        app_file_listed.append(['aaaaaaa',num_of_pgs])
        
        next_record = DataFrame(app_file_listed, columns=['word',f_col])
        
        #next_record = DataFrame(get_sorted_count(
        #    normalize(str(app_file_text))), columns=['word',f_col])
        
        # next_record.append(['0pages',num_of_pgs])
        
        # print(next_record)
        
        # vars()[str.replace(file, '.', '')] = DataFrame(
        #    get_sorted_count(normalize(str(app_file_text))), 
        #            columns=['word',f_col])
        
        #all_apps_df = pd.merge(
        #    all_apps_df, vars()[str.replace(
        #        file, '.', '')], on='word', how='outer')
        
        all_apps_df = pd.merge(
            all_apps_df, next_record, on='word', how='outer')
    else:
        print('f')
        error_log.append('Longer than 19 pgs : ' + os.path.join(
            'pprapps', file))


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

aacc.edu0.pdf
a


c
e



In [65]:
all_apps_df.fillna(0)[all_apps_df['word'] == 'aaaaaaa']

,fq0,word,aacc.edu0
1152,0,aaaaaaa,12


In [ ]:
error_log

In [ ]:
list_of_isApp_df = pd.read_csv(
    os.path.join('app_rec_train', 'list_of_isApp.csv'))

In [ ]:
lengths = []
for app in list_of_isApp_df['rootdom']:
    try:
        # print(os.path.join('pprapps', app + '.pdf'))
        app_c = PyPDF2.PdfFileReader(os.path.join('pprapps', app + '.pdf'))
        lengths.append(app_c.numPages)
        # lengths.append(app)
    except FileNotFoundError:
        print('{} : {}'.format(
                'Could not find', os.path.join('pprapps', app + '.pdf')))

In [ ]:
list_of_isApp_df.head()

In [ ]:
print (max(lengths))